## Notebook for hacking on migrating gimmeSAD internals to numpy

In [7]:
import numpy as np
import os

workdir = "/Volumes/WorkDrive/gimmeSAD"
os.chdir(workdir)

import Community



In [8]:
data = Community.Community()
data.set_metacommunity("metacommunity_LS4.txt")
data.prepopulate(mode="landbridge")

('total_inds', 1000300302)
('metacommunity', array([(0, 517471, 0.0005173156490759512), (1, 66, 6.598018601817837e-08),
       (2, 1, 9.996997881542177e-10), ...,
       (1634, 21854754, 0.021848192943962542),
       (1635, 138741, 0.0001386993483083043),
       (1636, 3255510, 0.0032545326573339374)], 
      dtype=[('uuid', '<i8'), ('abund', '<i8'), ('col_prob', '<f8')]))
[('uuid', '<i8'), ('abund', '<i8'), ('col_prob', '<f8')]
('init_community', array([[  2,   0,   0, ..., 247,   0,  24]]))
Initializing local community:
N species = 477
N individuals = 10000
('local_community', array([(0, 2, 1, True), (1, 0, 0, False), (2, 0, 0, False), ...,
       (1634, 247, 1, True), (1635, 0, 0, False), (1636, 24, 1, True)], 
      dtype=[('uuid', '<i8'), ('abund', '<i8'), ('col_time', '<i8'), ('is_founder', '?')]))


In [9]:
victim = np.random.multinomial(1, data.local_community["abund"]/float(data.K), size=1)
victim[0]
data.local_community["uuid"][victim[0].astype("bool")]

array([556])

In [61]:
#new_species = data.metacommunity["uuid"][migrant_draw.astype("bool")]
#print(migrant_draw, new_species)
wat = []
allsp = []
%timeit [allsp.extend([x] * x) for x in data.metacommunity["uuid"]]; wat = random.choice(allsp)
#allsp = np.array(allsp)
#%timeit migrant_draw = np.random.multinomial(10000, data.metacommunity["col_prob"], size=1)[0]
#%timeit random.choice(allsp)
print(len(allsp))
print(random.choice(allsp))

10 loops, best of 3: 18.4 ms per loop
54901706
1443


In [37]:
import random
%timeit random.choice(data.metacommunity["uuid"])
%timeit np.random.choice(data.metacommunity["uuid"])

10000 loops, best of 3: 22.6 µs per loop


In [65]:
source_pool = []
[source_pool.extend([x] * y) for x, y in zip(data.metacommunity["uuid"], data.metacommunity["abund"])]
print(random.choice(source_pool))
print(len(source_pool))

614
1000300302


In [112]:
wat = np.random.rand(100)
wat2d = np.zeros((100,2), dtype=int)
#print(wat2d)
a = np.recarray(100, dtype=[('wat', np.int)])
b = np.array([0]*100, dtype=np.int)
#%timeit wat[5] += 1
#%timeit np.add.at(wat, [5], 1)
#%timeit wat2d[5][1] += 1
#%timeit wat[5] -= 1
%timeit b[5]
%timeit a['wat'][5]
dict_homo = {'a': np.zeros(10000), 'b': np.zeros(10000)}
np_homo = np.zeros(10000, dtype=[('a', np.double), ('b', np.double)])
#%timeit dict_homo['a'][0] += 1
%timeit np_homo["a"][0] += 1

The slowest run took 18.86 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 265 ns per loop
100000 loops, best of 3: 14.6 µs per loop
The slowest run took 13.24 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 919 ns per loop


In [6]:
a = np.zeros(100, dtype=[('wat', np.int64)])
b = np.zeros(100, dtype=np.int64)
%timeit a['wat'][5] += 1
%timeit b[5] += 1


The slowest run took 16.37 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 845 ns per loop
The slowest run took 7.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 629 ns per loop


In [263]:
## 10000 loops, best of 3: 165 µs per loop
def rand1(data):
    idx = (np.cumsum(data.local_community[np.nonzero(data.local_community["abund"])]["abund"])/float(data.K)).searchsorted(np.random.sample(1))[0]
    local_abund = np.nonzero(data.local_community["abund"])[0]
    #print(idx)#, len(local_abund), local_abund)
    idx = np.nonzero(data.local_community["abund"])[0][idx]
    #print(idx)
    return idx

## 10000 loops, best of 3: 91.6 µs per loop
def rand2(data):
    victim = np.random.multinomial(1, data.local_community["abund"]/float(data.K), size=1)
    idx = data.local_community['uuid'][victim[0].astype("bool")]
    return idx

import itertools, random

## 1000 loops, best of 3: 1.94 ms per loop
def rand3(data):
    local_abunds = [[x[0]] * x[1] for x in data.local_community[np.nonzero(data.local_community["abund"])]]
    ## flatten list
    local_abunds = list(itertools.chain.from_iterable(local_abunds))
    idx = random.choice(local_abunds)

## 10000 loops, best of 3: 118 µs per loop
def rand4(data):
    subset = data.local_community[np.nonzero(data.local_community["abund"])]
    lt = subset["abund"].cumsum()
    get = lt.searchsorted(random.randint(0, lt.max()))
    return get

## 10000 loops, best of 3: 105 µs per loop
## 10000 loops, best of 3: 33.8 µs per loop
def rand5(data):
    #idx = data.local_community[np.nonzero(data.local_community["abund"])]["abund"].cumsum().searchsorted(random.randint(0, data.K))
    idx = np.nonzero(data.local_community["abund"])[0].cumsum().searchsorted(random.randint(0, data.K))
    return idx

In [266]:
%timeit rand5(data)

The slowest run took 8.48 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 33.8 µs per loop


In [247]:
subset = data.local_community[np.nonzero(data.local_community["abund"])]
lt = subset["abund"].cumsum()
get = lt.searchsorted(random.randint(0, lt.max()))
print(len(lt), lt.max(), get, lt, lt[get])

(477, 10000, 165, array([    2,    56,   182,   300,   311,   322,   323,   324,   342,
         346,   348,   358,   364,   373,   402,   404,   418,   461,
         572,   585,   657,   659,   660,   694,   695,   703,   709,
         711,   716,   717,   719,   720,   745,   814,   816,   817,
         819,   821,   832,   838,   842,   898,   951,   957,   961,
        1038,  1041,  1042,  1053,  1057,  1058,  1072,  1082,  1086,
        1088,  1097,  1166,  1178,  1187,  1189,  1202,  1203,  1220,
        1228,  1248,  1255,  1256,  1353,  1359,  1360,  1416,  1434,
        1435,  1436,  1437,  1449,  1461,  1474,  1475,  1485,  1486,
        1657,  1676,  1723,  1771,  1790,  1871,  1872,  1880,  1888,
        1892,  1893,  1894,  1905,  1927,  1953,  1954,  1955,  1956,
        1964,  1985,  2019,  2021,  2039,  2046,  2053,  2057,  2123,
        2131,  2135,  2136,  2219,  2232,  2234,  2294,  2302,  2500,
        2518,  2523,  2565,  2567,  2614,  2615,  2620,  2765,  2769,
  

In [274]:
random.choice([1,2,3,4,4,4,4,4,5,5,5,5,5,5,5])

5